In [ ]:
# prompt: # take the data set from internet eng-fench translation

import pandas as py
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
from tensorflow import keras
import os
from pathlib import Path

!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip spa-eng.zip


--2024-07-21 03:57:18--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.207, 173.194.193.207, 142.250.152.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.007s  

2024-07-21 03:57:18 (372 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]

Archive:  spa-eng.zip
   creating: spa-eng/
  inflating: spa-eng/_about.txt      
  inflating: spa-eng/spa.txt         


In [ ]:
# read the dataset into pandas data frame
df = py.read_csv('spa-eng/spa.txt', names=['eng', 'sap'], sep='\t')
df.head()


,eng,sap
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [ ]:
df.tail()

,eng,sap
118959,There are four main causes of alcohol-related ...,Hay cuatro causas principales de muertes relac...
118960,There are mothers and fathers who will lie awa...,Hay madres y padres que se quedan despiertos d...
118961,A carbon footprint is the amount of carbon dio...,Una huella de carbono es la cantidad de contam...
118962,Since there are usually multiple websites on a...,Como suele haber varias páginas web sobre cual...
118963,"If you want to sound like a native speaker, yo...","Si quieres sonar como un hablante nativo, debe..."


In [ ]:
# Configuration
batch_size = 64
epochs = 50
latent_dim = 256
num_samples = 10000

In [ ]:
# prepaing the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
# read the input and target texts from the data frame where coloums are eng and spa
for _, row in df.iterrows():
  input_text = row['eng']
  target_text = row['sap']
  # append the input and target texts to the list
  input_texts.append(input_text)
  target_texts.append(target_text)
  # append the unique characters to the set
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  # append the unique characte to the set
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
print(input_texts)
print(target_texts)



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 118964
Number of unique input tokens: 90
Number of unique output tokens: 111
Max sequence length for inputs: 247
Max sequence length for outputs: 278


In [ ]:
#index the token
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, '+': 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'á': 77, 'ã': 78, 'è': 79, 'é': 80, 'ê': 81, 'ó': 82, 'ö': 83, 'ü': 84, 'č': 85, '‘': 86, '’': 87, '₂': 88, '€': 89}
{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '

In [ ]:
#encoder input data
encoder_input_data=np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
encoder_input_data.shape

(118964, 247, 90)

In [ ]:
#decoder inputs and outputs
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')


In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
# building the model
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim,return_state=True)(encoder_inputs)
enccoder_outputs, state_h, state_c = keras.layers.LSTM(latent_dim,return_state=True)(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/50
1488/1488 [==============================] - 1430s 959ms/step - loss: 0.2101 - accuracy: 0.9369 - val_loss: 0.4171 - val_accuracy: 0.8708
Epoch 2/50
1488/1488 [==============================] - 1427s 959ms/step - loss: 0.1909 - accuracy: 0.9416 - val_loss: 0.3872 - val_accuracy: 0.8794
Epoch 3/50
1488/1488 [==============================] - 1427s 959ms/step - loss: 0.1776 - accuracy: 0.9456 - val_loss: 0.3954 - val_accuracy: 0.8799
Epoch 4/50
1488/1488 [==============================] - 1426s 958ms/step - loss: 0.1684 - accuracy: 0.9486 - val_loss: 0.3654 - val_accuracy: 0.8884
Epoch 5/50
1488/1488 [==============================] - 1426s 958ms/step - loss: 0.1607 - accuracy: 0.9509 - val_loss: 0.3429 - val_accuracy: 0.8958
Epoch 6/50
1488/1488 [==============================] - 1426s 959ms/step - loss: 0.1548 - accuracy: 0.9528 - val_loss: 0.3425 - val_accuracy: 0.8949
Epoch 7/50
1488/1488 [==============================] - 1427s 959ms/step - loss: 0.1498 - accuracy: 0.9543

KeyboardInterrupt: 